In [128]:
import numpy as np
import pandas as pd
from itertools import combinations
import time

In [68]:
df_ACM = pd.read_csv("ACM.csv")
df_DB = pd.read_csv("DBLP2.csv", encoding="latin1")

In [69]:
# 1. Concatenate all columns per row into one string
df_DB["merged"] = df_DB.astype(str).agg(" ".join, axis=1)
df_ACM["merged"] = df_ACM.astype(str).agg(" ".join, axis =1)

In [70]:
# 2. Change all alphabetical characters into lowercase.
df_DB["merged"] = df_DB["merged"].str.lower()
df_ACM["merged"] = df_ACM["merged"].str.lower()

In [71]:
# 3. Convert multiple spaces to one.
df_DB["merged"] = df_DB["merged"].str.replace(r"\s+", " ", regex=True).str.strip()
df_ACM["merged"] = df_ACM["merged"].str.replace(r"\s+", " ", regex=True).str.strip()


In [102]:
# 4. Combine the records from both tables into one big list as we did during the lab.
list_ACM = df_ACM["merged"].tolist()
list_DB  = df_DB["merged"].tolist()

big_list = list_ACM + list_DB
print(f"Total length of big_list: {len(big_list)}")  # Should be 4910

Total length of big_list: 4910


In [122]:
# 5. Use the functions in the tutorials from lab 5 to compute the shingles, the minhash signature and the similarity.
def shingle(text, k: int) -> set:
    """Return set of k-shingles, robust to None/NaN/non-strings/short strings."""
    # Treat None/NaN as empty
    if text is None:
        return set()
    if isinstance(text, float):
        if text != text:  # NaN
            return set()
        text = str(text)
    elif not isinstance(text, str):
        text = str(text)

    if len(text) < k:
        return set()
    return { text[i:i+k] for i in range(len(text) - k + 1) }

def build_vocab(shingle_sets: list) -> dict:
    full_set = {sh for s in shingle_sets for sh in s}
    return {sh: i for i, sh in enumerate(full_set)}

def one_hot(shingles: set, vocab: dict):
    vec = np.zeros(len(vocab), dtype=int)
    for sh in shingles:
        vec[vocab[sh]] = 1
    return vec

def get_minhash_arr(num_hashes:int, vocab:dict):
    length = len(vocab)
    arr = np.zeros((num_hashes, length), dtype=int)
    for i in range(num_hashes):
        arr[i, :] = np.random.permutation(length) + 1
    return arr

def get_signature(minhash: np.ndarray, vector: np.ndarray):
    idx = np.nonzero(vector)[0]
    if idx.size == 0:
        # No shingles; return a signature that won't match others
        return np.full(minhash.shape[0], np.iinfo(np.int32).max, dtype=int)
    return np.min(minhash[:, idx], axis=1)

def jaccard_similarity(set1: set, set2: set) -> float:
    inter = len(set1 & set2)
    union = len(set1 | set2)
    return inter / union if union else 0.0

def compute_signature_similarity(sig1: np.ndarray, sig2: np.ndarray) -> float:
    if sig1.shape != sig2.shape:
        raise ValueError("Signature shapes must match.")
    return float(np.mean(sig1 == sig2))

# Shingling
k = 3
shingle_sets = [shingle(doc, k) for doc in big_list]

# Vocab & one-hot
vocab = build_vocab(shingle_sets)
if len(vocab) == 0:
    raise ValueError("Vocabulary is empty. Check that big_list has strings of length >= k.")

onehot = np.stack([one_hot(sset, vocab) for sset in shingle_sets])

# MinHash signatures
num_hashes = 100
minhash_arr = get_minhash_arr(num_hashes, vocab)
signatures = np.stack([get_signature(minhash_arr, vec) for vec in onehot])

#  similarities
N = len(big_list)
jac_mat = np.eye(N, dtype=float) # exact Jaccard similarity matrix
mh_mat  = np.eye(N, dtype=float)  # MinHash similarity matrix

for i in range(N):
    for j in range(i + 1, N):
        # Exact Jaccard on shingles
        s_jac = jaccard_similarity(shingle_sets[i], shingle_sets[j])
        jac_mat[i, j] = jac_mat[j, i] = s_jac

        # MinHash-based similarity (fraction of equal signature components)
        s_mh = compute_signature_similarity(signatures[i], signatures[j])


        mh_mat[i, j] = mh_mat[j, i] = s_mh


In [119]:
# print shingles
print("\n=== Example shingles (first 3 documents) ===")
for i, s in enumerate(shingle_sets[:3]):
    print(f"Doc {i}: {sorted(list(s))[:10]} ... ({len(s)} total shingles)")

# Print MinHash signatures
print("\n=== Example MinHash signatures (first 3 documents) ===")
for i, sig in enumerate(signatures[:3]):
    print(f"Doc {i} signature: {sig[:10]} ... ({len(sig)} total hash values)")

# Print Jaccard & MinHash similarities
print("\n=== Jaccard similarity matrix (rounded) ===")
print(np.round(jac_mat, 3))

print("\n=== MinHash similarity matrix (rounded) ===")
print(np.round(mh_mat, 3))



=== Example shingles (first 3 documents) ===
Doc 0: [' 19', ' co', ' da', ' go', ' in', ' ma', ' ob', ' of', ' on', ' sy'] ... (126 total shingles)
Doc 1: [' 19', ' a ', ' co', ' cr', ' da', ' di', ' f.', ' fo', ' in', ' is'] ... (147 total shingles)
Doc 2: [' 19', ' an', ' at', ' be', ' bo', ' co', ' da', ' he', ' in', ' ja'] ... (188 total shingles)

=== Example MinHash signatures (first 3 documents) ===
Doc 0 signature: [ 18 183  46  11 268  23  13 149  28  86] ... (140 total hash values)
Doc 1 signature: [ 18 358  46  11  12  23 172 225 141  67] ... (140 total hash values)
Doc 2 signature: [ 18  27  46  11  12  15 142 112  98 178] ... (140 total hash values)

=== Jaccard similarity matrix (rounded) ===
[[1.    0.252 0.256 ... 0.054 0.016 0.046]
 [0.252 1.    0.255 ... 0.098 0.019 0.083]
 [0.256 0.255 1.    ... 0.093 0.032 0.074]
 ...
 [0.054 0.098 0.093 ... 1.    0.071 0.082]
 [0.016 0.019 0.032 ... 0.071 1.    0.037]
 [0.046 0.083 0.074 ... 0.082 0.037 1.   ]]

=== MinHash simila

In [132]:
""" 6. Extract the top 2224 candidates from the LSH algorithm, compare them to the actual
    mappings in the file DBLP-ACM_perfectMapping.csv
    and compute the precision of the method.
    7. Record the running time of the method.
    8. Compare the precision and the running time in Parts 1 and 2."""

start_time = time.time()
perfect_mapping_df = pd.read_csv("DBLP-ACM_perfectMapping.csv")
# the following code is taken from lab 5
class LSH:
    """
    Implements the Locality Sensitive Hashing (LSH) technique for approximate
    nearest neighbor search.
    """
    buckets = []
    counter = 0

    def __init__(self, b: int):
        """
        Initializes the LSH instance with a specified number of bands.

        Parameters:
        - b (int): The number of bands to divide the signature into.
        """
        self.b = b
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature: np.ndarray) -> np.ndarray:
        """
        Divides a given signature into subvectors based on the number of bands.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be divided.

        Returns:
        - np.ndarray: A stacked array where each row is a subvector of the signature.
        """
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)

    def add_hash(self, signature: np.ndarray):
        """
        Adds a signature to the appropriate LSH buckets based on its subvectors.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be hashed and added.
        """
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def check_candidates(self) -> set:
        """
        Identifies candidate pairs from the LSH buckets that could be potential near duplicates.

        Returns:
        - set: A set of tuple pairs representing the indices of candidate signatures.
        """
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)

# 25 bands gives threshold around 0.7 similarity
n_buckets = 25
lsh = LSH(n_buckets)

for signature in signatures:
    lsh.add_hash(signature)

candidate_pairs = lsh.check_candidates()

# built a list that gives each dataset an index position
len_acm = len(df_ACM)
index_to_id = []
for i in range(len_acm):
    record_id = df_ACM["id"].iloc[i]
    index_to_id.append(("ACM", record_id))
for i in range(len(df_DB)):
    record_id = df_DB["id"].iloc[i]
    index_to_id.append(("DBLP", record_id))

# keep only the pairs where the two documents come from different sources
cross_source_pairs = [(i, j) for (i, j) in candidate_pairs
                      if index_source_id[i][0] != index_source_id[j][0]]

#  Score them using MinHash signature similarity
scored = []
for i, j in cross_source_pairs:
    sim = compute_signature_similarity(signatures[i], signatures[j])
    scored.append((sim, i, j))

#  Sort by similarity and take top 2224
scored.sort(reverse=True, key=lambda x: x[0])
top_n = scored[:2224]

#  create set of predicted pairs
pred_pairs = set()
for sim, i, j in top_n:
    id_acm = str(index_to_id[i][1])
    id_dblp = str(index_to_id[j][1])
    pred_pairs.add((id_dblp, id_acm))

true_pairs = set(zip(perfect_mapping_df["idDBLP"].astype(str),
                     perfect_mapping_df["idACM"].astype(str)))

#  Compute precision
tp = len(pred_pairs.intersection(true_pairs))
fp = len(pred_pairs) - tp
precision = tp / len(pred_pairs) if pred_pairs else 0.0


print("precision =", round(precision, 4))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n---")
print("running time:", round(elapsed_time, 2), "seconds")

precision = 0.8251

---
running time: 2.53 seconds
